In [9]:
%matplotlib ipympl

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import gsw
import matplotlib.dates as mdates
import utide

In [10]:
ds = xr.open_dataset('data/MBARI_StationM_ADCP_201711_201811.nc')

In [11]:
ds

<xarray.Dataset>
Dimensions:    (beam: 4, bin: 23, time: 98208)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-10T10:03:13.760000 ... 2018-10-17T09:58:13.720000
Dimensions without coordinates: beam, bin
Data variables:
    Pit        (time) float64 ...
    Rol        (time) float64 ...
    Hea        (time) float64 ...
    Tem        (time) float64 ...
    Dep        (time) float64 ...
    Ori        (time) int64 ...
    BIT        (time) int64 ...
    Bat        (time) int64 ...
    Eas        (time, bin) float64 ...
    Nor        (time, bin) float64 ...
    Ver        (time, bin) float64 ...
    Err        (time, bin) float64 ...
    EA         (time, bin, beam) float64 ...
    PG         (time, bin, beam) float64 ...
    C          (time, bin, beam) float64 ...
    binheight  (bin) float64 ...
    Eas_tide   (time, bin) float64 ...
    Nor_tide   (time, bin) float64 ...
    Eas_filt   (time, bin) float64 ...
    Nor_filt   (time, bin) float64 ...
Attributes:
    PingsPer

In [12]:
# constit=['M2','S2','N2','K2','K1','O1','P1','Q1']

# constit_list=['M2', 'S2', 'K1', 'N2', 'O1']

constit_list = ['M2', 'S2', 'K1', 'N2', 'O1', 'MM', 'SSA', 'K2', 'P1', 'MSF',
        'MSM', 'NU2', 'OQ2', 'LDA2', '2N2', 'L2', 'MKS2', 'MF', 'M4',
        'UPS1', 'EPS2', 'Q1', 'BET1', 'THE1', 'MU2', 'MK3', 'MSN2', 'MS4',
        'MO3', 'MK4', 'J1', 'ETA2', '2SK5', 'MN4', 'NO1', 'SN4', 'TAU1',
        '2MN6', 'SK3', 'SK4', 'CHI1', '2MK5', 'S4', 'SIG1', 'M3', 'PHI1',
        'RHO1', 'SO3', 'OO1', '2SM6', 'ALP1', '2MS6', 'MSK6', 'SO1', 'M6',
        'M8', '2MK6', '3MK7', '2Q1']

height = ds['binheight'] + 0.4572 # add instrument height (18 inches)

Lsmaj = np.nan*np.ones([len(height),len(constit_list)])
Lsmin = np.nan*np.ones([len(height),len(constit_list)])
theta = np.nan*np.ones([len(height),len(constit_list)])
g = np.nan*np.ones([len(height),len(constit_list)])
SNR = np.nan*np.ones([len(height),len(constit_list)])
PE = np.nan*np.ones([len(height),len(constit_list)])

Lsmaj_ci = np.nan*np.ones([len(height),len(constit_list)])
Lsmin_ci = np.nan*np.ones([len(height),len(constit_list)])
theta_ci = np.nan*np.ones([len(height),len(constit_list)])
g_ci = np.nan*np.ones([len(height),len(constit_list)])

# tidal analysis
time = mdates.date2num(ds['time'])
for zi in np.arange(len(ds['binheight'])):
    print('bin '+str(zi+1)+'/'+str(len(ds['binheight'])))
    coef = utide.solve(time, ds['Eas'][:,zi], ds['Nor'][:,zi],
             lat=35+8.4585/60,
             constit=constit_list,
             method='ols')
    
    ni = np.argsort(coef['aux']['frq'])
    
    print(coef['name'][ni])
    
    Lsmaj[zi,:] = coef['Lsmaj'][ni]
    Lsmin[zi,:] = coef['Lsmin'][ni]
    theta[zi,:] = coef['theta'][ni]
    g[zi,:] = coef['g'][ni]
    SNR[zi,:] = coef['diagn']['SNR'][ni]
    PE[zi,:] = coef['diagn']['PE'][ni]
    Lsmaj_ci[zi,:] = coef['Lsmaj_ci'][ni]
    Lsmin_ci[zi,:] = coef['Lsmin_ci'][ni]
    theta_ci[zi,:] = coef['theta_ci'][ni]
    g_ci[zi,:] = coef['g_ci'][ni]

bin 1/23
solve: matrix prep ... solution ... 

KeyboardInterrupt: 

In [5]:
dstide = xr.Dataset({'Lsmaj': (('height', 'constituent'), Lsmaj),
                     'Lsmin': (('height', 'constituent'), Lsmin),
                     'theta': (('height', 'constituent'), theta),
                     'g': (('height', 'constituent'), g),
                     'SNR': (('height', 'constituent'), SNR),
                     'PE': (('height', 'constituent'), PE),
                     'Lsmaj_ci': (('height', 'constituent'), Lsmaj_ci),
                     'Lsmin_ci': (('height', 'constituent'), Lsmin_ci),
                     'theta_ci': (('height', 'constituent'), theta_ci),
                     'g_ci': (('height', 'constituent'), g_ci),
                     'freq':(('constituent'),coef['aux']['frq'][ni])},
                {'height': np.array(height), 'constituent': coef['name'][ni]})

In [6]:
dstide.attrs['reftime']= coef['aux']['reftime']

In [7]:
dstide.to_netcdf('utide_results_5const.nc')

In [8]:
coef

{'name': array(['M2', 'S2', 'K1', 'O1', 'N2', 'LDA2', 'NO1', 'MSM', 'Q1', '2Q1',
        'MM', 'ALP1', 'M6', 'NU2', 'SO1', 'MS4', 'SN4', 'MKS2', 'OO1',
        'L2', 'PHI1', 'UPS1', 'SK4', 'SIG1', 'MO3', 'S4', 'SK3', 'SSA',
        'MSK6', '2N2', 'MN4', 'P1', 'M3', 'MU2', 'J1', 'K2', 'RHO1', 'M4',
        'MF', '2MK5', 'MSN2', 'M8', 'MK3', 'TAU1', '2SM6', 'MK4', 'THE1',
        'EPS2', 'MSF', 'BET1', 'CHI1', '2MK6', 'SO3', '2MN6', '2MS6',
        'ETA2', '2SK5', 'OQ2', '3MK7'], dtype=object),
 'aux': {'frq': array([8.05114007e-02, 8.33333333e-02, 4.17807462e-02, 3.87306544e-02,
         7.89992487e-02, 8.18211814e-02, 4.02685942e-02, 1.30978069e-03,
         3.72185025e-02, 3.57063505e-02, 1.51215197e-03, 3.43965698e-02,
         2.41534202e-01, 7.92016200e-02, 4.46026789e-02, 1.63844734e-01,
         1.62332582e-01, 8.07395598e-02, 4.48308380e-02, 8.20235526e-02,
         4.20089053e-02, 4.63429900e-02, 1.66894826e-01, 3.59087218e-02,
         1.19242055e-01, 1.66666667e-01, 1.2511408